In [10]:
import numpy as np
import itertools
import torch

from konlpy.tag import Mecab
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

def get_candidates(text, module):
    if module == "okt":
        okt = Okt()
        tokenized_doc = okt.pos(text)
        tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])
    elif module == "mecab":
        mecab = Mecab()
        tokenized_doc = mecab.pos(text)
        tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] in ['NNP','NNG','SL']])
    else:
        raise ValueError("no module")
    
    n_gram_range = (1,1)
    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()
    return candidates


def dist_keywords(doc_embedding, candidate_embeddings, candidates, top_n):
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    return keywords


def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim
    if candidate:
        keywords = [words_vals[idx] for idx in candidate]
        return keywords
    else:
        return None


def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)
    
    if keywords_idx:
        keywords = [words[idx] for idx in keywords_idx]
        return keywords
    else:
        return None

def get_keyword(text, top_n, module, model):
    candidates = get_candidates(text, module)
    doc_embedding = model.encode([text])
    candidate_embeddings = model.encode(candidates)
    
    results = list()
    results.append(dist_keywords(doc_embedding, candidate_embeddings, candidates, top_n=top_n))
    results.append(max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=top_n, nr_candidates=top_n*2))
    results.append(mmr(doc_embedding, candidate_embeddings, candidates, top_n=top_n, diversity=0.8))
    
    return results

In [11]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
_ = model.eval()

In [16]:
text = "안녕하십니까? 문체부 대변인입니다. 1월 28일 월요일 정례브리핑을 시작하겠습니다. 금일 브리핑 순서는 주요 장차관 일정, 두 번째 보도자료 배포계획, 세 번째 현안, 질의·응답순으로 진행토록 그렇게 하겠습니다. 금주 장차관 주요일정입니다. 먼저, 장관입니다. 내일 제4회 국무회의에 참가하시고요. 수요일에는 제20차 장애인정책조정위원회에 참석하십니다. 그리고 31일 목요일에는 콘텐츠산업 청년종사자 간담회를 천안에서 개최할 계획입니다. 1차관 관련 부분입니다. 수요일에 제2차 생활SOC협의회에 참석하시고, 2차관은 1월 28일 금일 설 명절 계기 민생현장 방문이 예정되어 있습니다. 금주 보도자료 배포계획입니다. 오늘 브리핑 계획은 1월 문화가 있는 날과 불법게임장 근절 관계기관 간담회 개최, 2건입니다. 이 부분은 잠시 후에 상세하게 말씀드리고. 이번 주 주요 보도자료는 화요일에는 ‘2018 국민여가활동조사’ 결과 발표가 나갈 예정이고, 수요일에는 ‘2019 문화누리카드 발급’ 관련 보도자료, 그다음에 국립박물관·미술관 설 명절 계기 문화행사 개최 계획이 안내될 예정입니다. 31일은 지역콘텐츠 진흥 종사자 간담회 개최 계획이 있고, ‘2018 광고산업 통계조사’ 결과가 발표될 예정입니다. 금요일에는 ‘2019 지자체 개최 국제경기대회 공모사업’ 결과가 발표될 예정입니다. 오늘 브리핑은 먼저, 2019년 첫 번째 문화가 있는 날 행사 계획 관련입니다."


results = get_keyword(text, 5, "mecab", model)
unique_keywords = []
for result in results:
    unique_keywords.extend(result)

str_keyword = ' '.join(map(str, list(set(unique_keywords))))
print("===== mecab 결과 =====")
print("Cosine Similarity :", results[0])
print("Max Sum Similarity :", results[1])
print("Maximal Marginal Relevance :", results[2])
print("total :",str_keyword)

===== mecab 결과 =====
Cosine Similarity : ['협의회', '계획', '회의', '간담회', '월요일']
Max Sum Similarity : ['화요일', '정책', '계획', '회의', '월요일']
Maximal Marginal Relevance : ['월요일', '사업', '계획', '대변인', '문화']
total : 계획 사업 정책 협의회 월요일 문화 회의 간담회 화요일 대변인
